In [1]:
# -*- coding: utf-8 -*-
"""
Copyright ©2017. The Regents of the University of California (Regents).
All Rights Reserved. Permission to use, copy, modify, and distribute this
software and its documentation for educational, research, and not-for-profit
purposes, without fee and without a signed licensing agreement, is hereby
granted, provided that the above copyright notice, this paragraph and the
following two paragraphs appear in all copies, modifications, and
distributions. Contact The Office of Technology Licensing, UC Berkeley, 2150
Shattuck Avenue, Suite 510, Berkeley, CA 94720-1620, (510) 643-7201,
otl@berkeley.edu,
http://ipira.berkeley.edu/industry-info for commercial licensing opportunities.

IN NO EVENT SHALL REGENTS BE LIABLE TO ANY PARTY FOR DIRECT, INDIRECT, SPECIAL,
INCIDENTAL, OR CONSEQUENTIAL DAMAGES, INCLUDING LOST PROFITS, ARISING OUT OF
THE USE OF THIS SOFTWARE AND ITS DOCUMENTATION, EVEN IF REGENTS HAS BEEN
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

REGENTS SPECIFICALLY DISCLAIMS ANY WARRANTIES, INCLUDING, BUT NOT LIMITED TO,
THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE. THE SOFTWARE AND ACCOMPANYING DOCUMENTATION, IF ANY, PROVIDED
HEREUNDER IS PROVIDED "AS IS". REGENTS HAS NO OBLIGATION TO PROVIDE
MAINTENANCE, SUPPORT, UPDATES, ENHANCEMENTS, OR MODIFICATIONS.

Script for training a Grasp Quality Convolutional Neural Network (GQ-CNN).

Author
------
Vishal Satish & Jeff Mahler
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import shutil
import time

from autolab_core import YamlConfig, Logger
import autolab_core.utils as utils
from gqcnn import get_gqcnn_model, get_gqcnn_trainer, utils as gqcnn_utils

/home/dof6/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
W0820 15:13:58.403717 140705360750336 deprecation_wrapper.py:119] From /home/dof6/.local/lib/python2.7/site-packages/gqcnn/utils/enums.py:44: The name tf.RunOptions is deprecated. Please use tf.compat.v1.RunOptions instead.

I0820 15:13:58.412033 140705360750336 driver.py:120] Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt


root       INFO     Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt


I0820 15:13:58.437899 140705360750336 driver.py:120] Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


root       INFO     Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


W0820 15:13:58.841063 140705360750336 kinect2_sensor.py:15] Unable to import pylibfreenect2. Python-only Kinect driver may not work properly.


...

root       WARNING  Failed to import ROS in phoxi_sensor.py. PhoXiSensor functionality unavailable.


I0820 15:13:59.244867 140705360750336 acceleratesupport.py:13] OpenGL_accelerate module loaded


OpenGL.acceleratesupport INFO     OpenGL_accelerate module loaded


I0820 15:13:59.247447 140705360750336 arraydatatype.py:270] Using accelerated ArrayDatatype


OpenGL.arrays.arraydatatype INFO     Using accelerated ArrayDatatype


In [2]:
if __name__ == "__main__":
    dataset_dir = "dynamic_grasp_data/training"
    split_name = "image_wise"
    output_dir = "grasp_learner/models/"
    tensorboard_port = None
    seed = 1
    config_filename = "grasp_experts/cfg/finetune_dex-net_4.0_pj.yaml"
    name = "FC-GQCNN-4.0-PJ-02"
    save_datetime = False
    backend = "tf"
    __file__ = os.path.abspath('')

    # Set default output dir.
    if output_dir is None:
        output_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                  "../models")

    # Set default config filename.
    if config_filename is None:
        config_filename = os.path.join(
            os.path.dirname(os.path.realpath(__file__)), "..",
            "cfg/train.yaml")

    # Turn relative paths absolute.
    if not os.path.isabs(dataset_dir):
        dataset_dir = os.path.join(os.getcwd(), dataset_dir)
    if not os.path.isabs(output_dir):
        output_dir = os.path.join(os.getcwd(), output_dir)
    if not os.path.isabs(config_filename):
        config_filename = os.path.join(os.getcwd(), config_filename)

    # Create output dir if necessary.
    utils.mkdir_safe(output_dir)

    # Open train config.
    train_config = YamlConfig(config_filename)
    if seed is not None:
        train_config["seed"] = seed
        train_config["gqcnn"]["seed"] = seed
    if tensorboard_port is not None:
        train_config["tensorboard_port"] = tensorboard_port
    gqcnn_params = train_config["gqcnn"]

    # Create a unique output folder based on the date and time.
    if save_datetime:
        # Create output dir.
        unique_name = time.strftime("%Y%m%d-%H%M%S")
        output_dir = os.path.join(output_dir, unique_name)
        utils.mkdir_safe(output_dir)

    
    # Set visible devices.
    if "gpu_list" in train_config:
        gqcnn_utils.set_cuda_visible_devices(train_config["gpu_list"])

#     train_config["num_epochs"] = 5
    
    # Train the network.
    start_time = time.time()
    gqcnn = get_gqcnn_model(backend)(gqcnn_params)
    trainer = get_gqcnn_trainer(backend)(gqcnn,
                                         dataset_dir,
                                         split_name,
                                         output_dir,
                                         train_config,
                                         name=name)
    trainer.train()
    shutil.rmtree(os.path.join(dataset_dir,"splits"), ignore_errors=True)

I0820 15:14:04.134243 140705360750336 __init__.py:61] Initializing GQ-CNN with Tensorflow as backend...


GQCNNModelFactory INFO     Initializing GQ-CNN with Tensorflow as backend...


I0820 15:14:04.137160 140705360750336 logger.py:57] Root logger now logging to /home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_learner/models/FC-GQCNN-4.0-PJ-02/training.log


root       INFO     Root logger now logging to /home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_learner/models/FC-GQCNN-4.0-PJ-02/training.log


I0820 15:14:04.139780 140705360750336 trainer_tf.py:1012] Saving model to: /home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_learner/models/FC-GQCNN-4.0-PJ-02


...

GQCNNTrainerTF INFO     Flushing prefetch queue...


I0820 15:15:26.155846 140705360750336 trainer_tf.py:230] Closing Tensorboard...


GQCNNTrainerTF INFO     Closing Tensorboard...


I0820 15:15:26.158773 140705360750336 network_tf.py:576] Closing TF Session...


GQCNNTF    INFO     Closing TF Session...
